_한국어로 기계번역됨_


# ReAct 에이전트에 스레드 수준 메모리 추가하기

<div class="admonition tip">
    <p class="admonition-title">전제 조건</p>
    <p>
        이 가이드는 다음에 대한 이해가 있다고 가정합니다:
        <ul>
            <li>            
                <a href="https://langchain-ai.github.io/langgraph/concepts/persistence/">
                    LangGraph 지속성
                </a>
            </li>
            <li>            
                <a href="https://langchain-ai.github.io/langgraph/concepts/persistence/#checkpointer-interface">
                    체크포인터 인터페이스
                </a>
            </li>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/">
                    에이전트 아키텍처
                </a>                   
            </li>
            <li>
                <a href="https://python.langchain.com/docs/concepts/chat_models/">
                    채팅 모델
                </a>
            </li>
            <li>
                <a href="https://python.langchain.com/docs/concepts/tools/">
                    도구
                </a>
            </li>
        </ul>
    </p>
</div> 

본 가이드는 사전 제작된 ReAct 에이전트에 메모리를 추가하는 방법을 보여줍니다. 사전 제작된 ReAct 에이전트 사용 시작 방법에 대해서는 [이 튜토리얼](../create-react-agent)을 참조하세요.

우리는 [create_react_agent](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent) 함수에 [체크포인터](https://langchain-ai.github.io/langgraph/reference/checkpoints/)를 전달함으로써 에이전트에 메모리를 추가할 수 있습니다.


## 설정

먼저, 필요한 패키지를 설치하고 API 키를 설정합시다.


In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 발견하고 성능을 향상시키세요. LangSmith는 여러분이 LangGraph로 구축한 LLM 앱을 디버깅, 테스트 및 모니터링하는 데 트레이스 데이터를 사용할 수 있게 해줍니다 — 시작하는 방법에 대한 더 많은 정보는 <a href="https://docs.smith.langchain.com">여기</a>에서 읽어보세요. 
    </p>
</div>


코드


In [3]:
# First we initialize the model we want to use.
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)


# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)

from typing import Literal

from langchain_core.tools import tool


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]

# We can add "chat memory" to the graph with LangGraph's checkpointer
# to retain the chat context between interactions
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# Define the graph

from langgraph.prebuilt import create_react_agent

graph = create_react_agent(model, tools=tools, checkpointer=memory)


## 사용법

여러 번 상호작용하여 기억할 수 있음을 보여줍시다.


In [5]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


In [6]:
config = {"configurable": {"thread_id": "1"}}
inputs = {"messages": [("user", "What's the weather in NYC?")]}

print_stream(graph.stream(inputs, config=config, stream_mode="values"))


================================ Human Message =================================

What's the weather in NYC?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_xM1suIq26KXvRFqJIvLVGfqG)
 Call ID: call_xM1suIq26KXvRFqJIvLVGfqG
  Args:
    city: nyc
================================= Tool Message =================================
Name: get_weather

It might be cloudy in nyc
================================== Ai Message ==================================

The weather in NYC might be cloudy.


같은 스레드 ID를 전달할 때 채팅 기록이 유지된다는 점에 유의하세요.


In [7]:
inputs = {"messages": [("user", "What's it known for?")]}
print_stream(graph.stream(inputs, config=config, stream_mode="values"))


================================ Human Message =================================

What's it known for?
================================== Ai Message ==================================

New York City (NYC) is known for a variety of iconic landmarks, cultural institutions, and vibrant neighborhoods. Some of the most notable aspects include:

1. **Statue of Liberty**: A symbol of freedom and democracy.
2. **Times Square**: Known for its bright lights, Broadway theaters, and bustling atmosphere.
3. **Central Park**: A large urban park offering a green oasis in the middle of the city.
4. **Empire State Building**: An iconic skyscraper with an observation deck offering panoramic views of the city.
5. **Broadway**: Famous for its world-class theater productions.
6. **Wall Street**: The financial hub of the United States.
7. **Museums**: Including the Metropolitan Museum of Art, the Museum of Modern Art (MoMA), and the American Museum of Natural History.
8. **Diverse Cuisine**: A melting pot o